In [74]:
import json
import requests
import praw
import numpy

reddit = praw.Reddit(client_id='tiBit4kBeuoOTA', 
                     client_secret='7-HPDa9q92MS0a-AnUVYb72QDzU', 
                     user_agent='TorkilSamStat', 
                     username='TorkSamStat', 
                     password='bbybbyduerinnafor')

# STATIC VARIABLES
base_url = 'https://api.pushshift.io/reddit/search/comment/'
wanted_count_auth = 4
wanted_count_sub = 4
unwanted_users = ["[deleted]", "AutoModerator"]
unwanted_count = len(unwanted_users)

    
# FUNCTIONS
def getKeysFromDict(data):
    users = []
    for obj in data:
        if obj['key'] not in unwanted_users:
            users.append(obj['key'])
    return users

def getMostActiveAuthorsInSubReddit(subreddit):
    #if subreddit in used_subreddits:
    #    return []
    #else:
    url = base_url + '?since=100d&subreddit={}&aggs=author&agg_size={}'.format(subreddit, wanted_count_auth + unwanted_count)
    response = requests.get(url).json()
    fetched_authors = getKeysFromDict(response['aggs']['author'])
    return fetched_authors

def getMostActiveSubRedditForAuthor(author):
    #if author in used_authors:
    #    return []
    #else:
    url = base_url + '?since=100d&author={}&aggs=subreddit&agg_size={}'.format(author, wanted_count_sub)
    response = requests.get(url).json()
    fetched_subreddits = getKeysFromDict(response['aggs']['subreddit'])
    for sub in used_subreddits:
            if (sub not in used_subreddits):
                used_subreddits.append(sub)
    return fetched_subreddits

In [75]:
%%time
# Global variables
used_subreddits = []
used_authors = []


def get_relations(relational_array, subreddit, wanted_subcount):
    
    if wanted_subcount == 0:
        return []
    
    # Add subreddit to used_subreddits
    used_subreddits.append(subreddit)
    
    # Retrieve users from subreddit
    users = getMostActiveAuthorsInSubReddit(subreddit)
    
    # Retrieve subs from active users in subreddit
    for user in users:
        if user not in used_authors:
            subs = getMostActiveSubRedditForAuthor(user)
            used_authors.append(user)
            
            # Enter new level
            for sub in subs:
                if sub != subreddit:
                    relational_array.append([subreddit, sub])
                if sub not in used_subreddits:
                    relational_array.extend(get_relations([], sub, wanted_subcount - 1))
                    
    return relational_array


rels = get_relations([], 'The_Donald', 3)
rels

CPU times: user 8.24 s, sys: 241 ms, total: 8.48 s
Wall time: 12min 55s


[['The_Donald', 'florida'],
 ['florida', 'Unemployment'],
 ['Unemployment', 'AmItheAsshole'],
 ['Unemployment', 'BlackPeopleTwitter'],
 ['Unemployment', 'personalfinance'],
 ['Unemployment', 'washingtondc'],
 ['Unemployment', 'politics'],
 ['Unemployment', 'smallbusiness'],
 ['Unemployment', 'quicken'],
 ['Unemployment', 'legaladviceofftopic'],
 ['Unemployment', 'Adulting'],
 ['Unemployment', 'stimuluscheck'],
 ['Unemployment', 'politics'],
 ['Unemployment', 'xboxone'],
 ['florida', 'personalfinance'],
 ['personalfinance', 'mintuit'],
 ['personalfinance', 'MagicArena'],
 ['personalfinance', 'pittsburgh'],
 ['personalfinance', 'Volvo'],
 ['personalfinance', 'nottheonion'],
 ['personalfinance', 'apple'],
 ['personalfinance', 'AskSF'],
 ['personalfinance', 'askportland'],
 ['personalfinance', 'sanfrancisco'],
 ['personalfinance', 'investing'],
 ['personalfinance', 'CreditCards'],
 ['personalfinance', 'Bogleheads'],
 ['florida', 'politics'],
 ['politics', 'SeaWA'],
 ['politics', 'HistoryMe

### 